In [10]:
import os
from pathlib import Path
from typing import List

# -------  LangChain & utils  -------
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Embeddings (sentence-transformers)
from langchain_huggingface import HuggingFaceEmbeddings

# LLM vía Hugging Face Transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain_community.llms import HuggingFacePipeline

In [11]:
# Carpeta con tus documentos (txt/pdf). Para el demo, crearemos unos .txt.
DOCS_DIR = Path("docs")
DOCS_DIR.mkdir(exist_ok=True)

def ensure_sample_docs():
    """Crea algunos documentos de ejemplo si la carpeta está vacía."""
    if any(DOCS_DIR.iterdir()):
        return

    (DOCS_DIR / "planes.txt").write_text(
        "Planes de Internet:\n"
        "- Plan 50 Mbps: ideal para hogares pequeños.\n"
        "- Plan 100 Mbps: uso intensivo, streaming HD.\n"
        "- Plan 300 Mbps: múltiples dispositivos y teletrabajo.\n"
        "Velocidad mínima garantizada: 25 Mbps en horarios no pico (8:00-18:00)."
    )

    (DOCS_DIR / "soporte.txt").write_text(
        "Soporte técnico:\n"
        "1) Reinicie el router por 30 segundos.\n"
        "2) Verifique cables y luz de Internet.\n"
        "3) Si sigue el problema, contáctenos con su correo o teléfono para validar la cuenta."
    )

    (DOCS_DIR / "contrato.txt").write_text(
        "Contrato del servicio:\n"
        "La cláusula 5 indica la velocidad mínima garantizada. Las interrupciones programadas serán notificadas.\n"
        "Para cambios de plan, el titular debe identificarse con correo o teléfono."
    )


In [12]:
def load_documents() -> List[Document]:
    """Carga archivos .txt (y fácilmente extensible a PDF) como Documentos LangChain."""
    docs: List[Document] = []
    for p in DOCS_DIR.glob("*.txt"):
        text = p.read_text(encoding="utf-8")
        docs.append(Document(page_content=text, metadata={"source": str(p)}))
    return docs

In [13]:
# Modelo de embeddings (rápido y bueno)
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

# Elige uno de estos LLMs (asegúrate de tener acceso/aceptar la licencia en HF):
# - "meta-llama/Llama-2-7b-chat-hf"  (requiere acceso en HF)
# - "google/gemma-2b-it"             (más liviano)
# - "meta-llama/Llama-3.1-8B-Instruct" (si tienes acceso)

# - datificate/gpt2-small-spanish
# - meta-llama/Llama-3.2-1B
# - HuggingFaceTB/SmolLM3-3B
LLM_MODEL_NAME = os.environ.get("RAG_LLM_MODEL", "bigscience/bloom-560m")

# Modo CPU por defecto; ajusta si tienes GPU
DEVICE = "cuda"  # "cpu" ó "cuda" si tienes GPU y drivers

In [14]:
def build_vectorstore(documents: List[Document]) -> FAISS:
    """Crea el índice FAISS a pHuggingFaceTB/SmolLM3-3Bartir de documentos con embeddings de sentence-transformers."""
    embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=600, chunk_overlap=80, separators=["\n\n", "\n", ". ", " "]
    )
    chunks = text_splitter.split_documents(documents)
    return FAISS.from_documents(chunks, embedding=embeddings)

In [15]:
ensure_sample_docs()
docs = load_documents()
vs = build_vectorstore(docs)

In [16]:
"""Carga el LLM con Transformers y lo envuelve para LangChain."""

print(f"Cargando LLM: {LLM_MODEL_NAME}")
bnb_config = BitsAndBytesConfig(load_in_8bit=True)  # int8 works on your GPU
tok = AutoTokenizer.from_pretrained(LLM_MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(LLM_MODEL_NAME, trust_remote_code=True).to(
    DEVICE
)

Cargando LLM: bigscience/bloom-560m


In [17]:
# Check that the model is on GPU
print(next(model.parameters()).device)

cuda:0


In [18]:
def load_llm() -> HuggingFacePipeline:
    gen = pipeline(
        "text-generation",
        model=model,
        tokenizer=tok,
        max_new_tokens=300,
        temperature=0.8,
        top_p=0.9,
        do_sample=True,
        device=0 if DEVICE == "cuda" else -1,
    )
    return HuggingFacePipeline(pipeline=gen)

In [19]:
llm = load_llm()

Device set to use cuda:0
/tmp/ipykernel_24106/4127914580.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=gen)


In [20]:
def build_rag_chain(vs: FAISS, llm: HuggingFacePipeline):
    """Crea una cadena RetrievalQA con un prompt controlado."""
    retriever = vs.as_retriever(search_kwargs={"k": 4})

    prompt_tmpl = PromptTemplate.from_template(
        (
            "Eres un asistente de soporte al cliente salvadoreño: responde claro, breve y humano.\n"
            "Usa SOLAMENTE la información proporcionada en el contexto.\n"
            "Si algo no aparece, di que no está en los documentos.\n\n"
            "Contexto:\n{context}\n\n"
            "Pregunta del usuario:\n{question}\n\n"
            "Respuesta (tuteo suave, tono cercano salvadoreño):"
        )
    )

    # RetrievalQA ya maneja pasar {context} con los docs recuperados
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt_tmpl},
        return_source_documents=True,
    )
    return qa

In [21]:
def ask(qa_chain, query: str):
    """Consulta al RAG y muestra respuesta + fuentes."""
    res = qa_chain({"query": query})
    answer = res["result"]
    sources = [d.metadata.get("source") for d in res.get("source_documents", [])]
    print("\n🗣️  Usuario:", query)
    print("\n🤖  Bot:", answer.strip())
    print("\n📚  Fuentes:", list(dict.fromkeys(sources)))  # únicas

In [22]:
qa = build_rag_chain(vs, llm)

In [24]:
# Pruebas
ask(qa, "¿Cuál es la velocidad mínima garantizada?")
ask(qa, "¿Qué pasos recomiendas para soporte técnico si no tengo internet?")
ask(qa, "¿Cuáles planes tienen y para qué uso sirven?")


🗣️  Usuario: ¿Cuál es la velocidad mínima garantizada?

🤖  Bot: Eres un asistente de soporte al cliente salvadoreño: responde claro, breve y humano.
Usa SOLAMENTE la información proporcionada en el contexto.
Si algo no aparece, di que no está en los documentos.

Contexto:
Contrato del servicio:
La cláusula 5 indica la velocidad mínima garantizada. Las interrupciones programadas serán notificadas.
Para cambios de plan, el titular debe identificarse con correo o teléfono.

Soporte técnico:
1) Reinicie el router por 30 segundos.
2) Verifique cables y luz de Internet.
3) Si sigue el problema, contáctenos con su correo o teléfono para validar la cuenta.

Planes de Internet:
- Plan 50 Mbps: ideal para hogares pequeños.
- Plan 100 Mbps: uso intensivo, streaming HD.
- Plan 300 Mbps: múltiples dispositivos y teletrabajo.
Velocidad mínima garantizada: 25 Mbps en horarios no pico (8:00-18:00).

Pregunta del usuario:
¿Cuál es la velocidad mínima garantizada?

Respuesta (tuteo suave, tono cercano 